In [10]:
import h5py
import json
import numpy as np
import os
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from torch.utils.data import Dataset, DataLoader
import torch

torch.manual_seed(1)

img_data = './data/img_data'
text_data = './data/text_data'

stop = set(stopwords.words('english'))
stop.remove('no')
stop.update(['?', '\'s', ',', '!', '.', '-', ':', '_', ''])


stemmer = SnowballStemmer("english")

In [9]:
stem = True
stopwords = True
df = pd.read_json(os.path.join(text_data, 'Easy/IR_test_easy.json')).T.sort_index().head(10)

ValueError: Expected object or value

In [35]:
s = df.apply(lambda x: pd.Series(x['dialog']),axis=1)\
        .stack()\
        .reset_index(level=1, drop=True)

In [36]:
q_result = s.apply(lambda q: 
                   (lambda x = q[0].split('?')[1].strip().lower(): x if x == 'yes' or x == 'no' else '')())
q_result = q_result.reset_index(drop=True)
q_bool = s.apply(lambda q: 
                 (lambda x = q[0].split('?')[1].strip(): True if x == 'yes' or x == 'no' else False)())
s.name = 'question'
q_result = pd.Series(q_result[q_bool.reset_index(drop=True)].reset_index(drop=True))\
                     .apply(lambda x: True if x == 'yes' else False)
q_result.name = 'question_result'
df = df.drop('dialog', axis=1).join(s)[q_bool].reset_index(drop=True)\
                              .join(q_result).dropna()\
                              .reset_index(drop= True).drop('img_list', axis=1).drop('target', axis=1)
df.question = df.question.apply(lambda x: x[0])
df.question = df.question.apply(lambda sentence: 
                                tuple(
                                    [(lambda x: stemmer.stem(x) if stem else x)(i)
                                     for i in word_tokenize(sentence.lower()) 
                                     if i not in stop or not stopwords]
                                )
                               )

In [37]:
df

,caption,img_list,target_img_id,question,question_result
0,"a plate of broccoli, tomatoes, cheese and berr...","[81964, 185247, 298360, 286685, 70508, 239483,...",81964,"(peopl, pictur, no)",False
1,"a plate of broccoli, tomatoes, cheese and berr...","[81964, 185247, 298360, 286685, 70508, 239483,...",81964,"(utensil, pictur, yes)",True
2,"a plate of broccoli, tomatoes, cheese and berr...","[81964, 185247, 298360, 286685, 70508, 239483,...",81964,"(napkin, pictur, no)",False
3,"a plate of broccoli, tomatoes, cheese and berr...","[81964, 185247, 298360, 286685, 70508, 239483,...",81964,"(see, beverag, pictur, no)",False
4,"a plate of broccoli, tomatoes, cheese and berr...","[81964, 185247, 298360, 286685, 70508, 239483,...",81964,"(see, applianc, pictur, no)",False
5,a black jacket standing next to snowboard with...,"[485550, 384916, 355462, 457587, 411225, 30546...",175410,"(peopl, shot, no)",False
6,a group of skiers in a line skiing cross country,"[471901, 212063, 109177, 425577, 194553, 53940...",81961,"(back, toward, camera, yes)",True
7,a group of skiers in a line skiing cross country,"[471901, 212063, 109177, 425577, 194553, 53940...",81961,"(see, other, photo, yes)",True
8,a group of dogs laying on top of a bed,"[214210, 346437, 475902, 111648, 468992, 44793...",111648,"(peopl, no)",False
9,a group of dogs laying on top of a bed,"[214210, 346437, 475902, 111648, 468992, 44793...",111648,"(window, behind, couch, no)",False


In [5]:
df = pd.read_json(os.path.join(text_data, 'Easy/IR_train_easy.json')).T.sort_index()

In [6]:
df.caption = df.caption.\
                apply(lambda sentence: tuple([stemmer.stem(i) for i in word_tokenize(sentence.lower()) if i not in stop]))
caption_df = df[['caption', 'target_img_id']].drop_duplicates().reset_index(drop=True)

In [7]:
s = df.apply(lambda x: pd.Series(x['dialog']),axis=1)\
            .stack()\
            .reset_index(level=1, drop=True)
q_bool = s.apply(lambda q: (lambda x = q[0].split('?')[1].strip(): True if x == 'yes' or x == 'no' else False)())

In [8]:
s = df.apply(lambda x: pd.Series(x['dialog']),axis=1)\
            .stack()\
            .reset_index(level=1, drop=True)
q_result = s.apply(lambda q: (lambda x = q[0].split('?')[1].strip().lower(): x if x == 'yes' or x == 'no' else '')())
q_result = q_result.reset_index(drop=True)
q_bool = s.apply(lambda q: (lambda x = q[0].split('?')[1].strip(): True if x == 'yes' or x == 'no' else False)())
s.name = 'question'
q_result = pd.Series(q_result[q_bool.reset_index(drop=True)].reset_index(drop=True))#[q_bool]
q_result.name = 'question_result'

In [9]:
df = df.drop('dialog', axis=1).join(s)[q_bool].reset_index(drop=True)\
     .join(q_result).dropna().reset_index(drop= True).drop('img_list', axis=1).drop('target', axis=1)

In [10]:
df.question = df.question.apply(lambda x: x[0])

In [11]:
df.question = df.question.\
                apply(lambda sentence: tuple([stemmer.stem(i) for i in word_tokenize(sentence.lower()) if i not in stop]))

In [12]:
question_df = df[['question', 'question_result', 'target_img_id']]

In [4]:
df

NameError: name 'df' is not defined

In [39]:
class TrainDataSet(Dataset):

    def __init__(self, complexity, captions, img_feat_file = 'IR_image_features.h5', 
                 img_map_file = 'IR_img_features2id.json', stem = True, stopwords = True,
                normalize = True):
        csv_file = '{}/IR_train_{}.json'.format(complexity.capitalize(), complexity)
        csv_path = os.path.join(text_data, csv_file)
        self.captions = captions
        if captions:
            self.df = self.preprocess_captions(csv_path, stem, stopwords)
        else:
            self.df = self.preprocess_questions(csv_path, stem, stopwords)
            
        self.img_features, self.mean, self.std, self.visual_feat_mapping = self.load_image_data(img_feat_file, img_map_file)
            

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        img_id = self.df.iloc[idx, 1]
        h5_id = self.visual_feat_mapping[str(img_id)]
        img_feat = self.img_features[h5_id]
        if self.captions:
            target = True
        else:
            target = self.df.iloc[idx, 2]
        # join text for batch processing (otherwise every words gets in a different tuple)
        sample = {'text': ' '.join(text), 'img_features': img_feat, 'target': target}
        return sample
    
    def preprocess_captions(self, csv_path, stem, stopwords):
        df = pd.read_json(csv_path).T.sort_index()
        df.caption = df.caption.apply(lambda sentence: 
                                tuple(
                                    [(lambda x: stemmer.stem(x) if stem else x)(i)
                                     for i in word_tokenize(sentence.lower()) 
                                     if i not in stop or not stopwords]
                                )
                               )
        return df[['caption', 'target_img_id']].drop_duplicates().reset_index(drop=True)

    def preprocess_questions(self, csv_path, stem, stopwords):
        # TODO: Remove head after testing
        df = pd.read_json(csv_path).T.sort_index().head(50)
        s = df.apply(lambda x: pd.Series(x['dialog']),axis=1)\
                .stack()\
                .reset_index(level=1, drop=True)
        q_result = s.apply(lambda q: 
                           (lambda x = q[0].split('?')[1].strip().lower(): x if x == 'yes' or x == 'no' else '')())
        q_result = q_result.reset_index(drop=True)
        q_bool = s.apply(lambda q: 
                         (lambda x = q[0].split('?')[1].strip().lower(): True if x == 'yes' or x == 'no' else False)())
        s.name = 'question'
        q_result = pd.Series(q_result[q_bool.reset_index(drop=True)]\
                             .reset_index(drop=True))\
                             .apply(lambda x: 1 if x == 'yes' else -1)
        q_result.name = 'question_result'
        df = df.drop('dialog', axis=1).join(s)[q_bool].reset_index(drop=True)\
                                      .join(q_result).dropna()\
                                      .reset_index(drop= True).drop('img_list', axis=1).drop('target', axis=1)
        df.question = df.question.apply(lambda x: x[0])
        df.question = df.question.apply(lambda sentence: 
                                tuple(
                                    [(lambda x: stemmer.stem(x) if stem else x)(i)
                                     for i in word_tokenize(sentence.lower()) 
                                     if i not in stop or not stopwords]
                                )
                               )
        df_augmented = df.copy()
        df_augmented.question_result = -df_augmented.question_result
        df_augmented.question = df_augmented.question.\
                                apply(lambda sent: sent[:-1] + ('yes', ) 
                                      if sent[-1] == 'no' else sent[:-1] + ('no', ))
        df = df.append(df_augmented, ignore_index = True)
        return df[['question', 'target_img_id', 'question_result']]
    
    def load_image_data(self, img_feat_file, img_map_file):
        img_features = np.asarray(h5py.File(os.path.join(img_data, img_feat_file), 'r')['img_features'])
        with open(os.path.join(img_data, img_map_file), 'r') as f:
             visual_feat_mapping = json.load(f)['IR_imgid2id']
        mean = img_features.mean(axis=0)
        std = img_features.std(axis=0)
        img_features = (img_features - mean)/std
        return img_features, mean, std, visual_feat_mapping
    
    

In [3]:
data_train_captions = TrainDataSet(complexity = 'easy', captions = True)
data_train_questions = TrainDataSet(complexity = 'easy', captions = False)

# data_val_captions = TextImageDataSet(split = 'val',  complexity = 'easy', captions = False)
# data_val_questions = TextImageDataSet(split = 'val',  complexity = 'easy', captions = True)

# data_test_captions = TextImageDataSet(split = 'test',  complexity = 'easy', captions = False)
# data_test_questions = TextImageDataSet(split = 'test',  complexity = 'easy', captions = True)

In [4]:
captions_batch_size = 128
questions_batch_size = 512

dataloader_train_captions = DataLoader(data_train_captions, batch_size=captions_batch_size,
                        shuffle=True, num_workers=4)
dataloader_train_questions = DataLoader(data_train_questions, batch_size=questions_batch_size,
                        shuffle=True, num_workers=4)

# dataloader_val_captions = DataLoader(data_val_captions, batch_size=1000,
#                         shuffle=False, num_workers=4)
# dataloader_val_questions = DataLoader(data_val_questions, batch_size=1000,
#                         shuffle=False, num_workers=4)

# dataloader_test_captions = DataLoader(data_test_captions, batch_size=1000,
#                         shuffle=False, num_workers=4)
# dataloader_test_questions = DataLoader(data_test_questions, batch_size=1000,
#                         shuffle=False, num_workers=4)

In [5]:
def read_dataset(caption_dataset, question_dataset, dictionary):
    captions = list(caption_dataset.df.caption)
    questions = list(question_dataset.df.question)
    all_sentences = captions + questions
    _ = [dictionary[word] for sentence in all_sentences for word in sentence]
    return dictionary

def preprocess_sentences(sentences, dictionary):
    max_len = 0
    resulting_sentences = []
    for sentence in sentences:
        sentence = sentence.split(' ')
        max_len = max(max_len, len(sentence))
        resulting_sentences.append([dictionary[word] for word in sentence])
    resulting_sentences = [sentence + ([UNK] * (max_len - len(sentence))) for sentence in resulting_sentences]
    return torch.LongTensor(resulting_sentences)

In [6]:
from collections import defaultdict

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<unk>"]

w2i = defaultdict(lambda: UNK, 
                  read_dataset(data_train_captions, data_train_questions, w2i))

nwords = len(w2i)

In [7]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, img_feat_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, img_feat_size)
        
    def forward(self, input_text):
        input_text = input_text.view(input_text.size(0), -1)
        x = self.embeddings(input_text)
        x = torch.sum(x, 1)
        return x
    

model = CBOW(nwords, 64, 2048)

print(model)

optimizer = optim.Adam(model.parameters())

CBOW (
  (embeddings): Embedding(7683, 2048)
)


In [12]:
from torch.autograd import Variable

def train(epoch, model, dataloader, batch_size = 128, cuda = False):
    if cuda:
        model = model.cuda()
        
    model.train()
    train_loss = 0.0

    for idx, sample in enumerate(dataloader):
        text_data = preprocess_sentences(sample['text'], w2i)
        img_feat = sample['img_features'] 
        target = sample['target'].float()
        if cuda:
            text_data, img_feat, target = text_data.cuda(), img_feat.cuda(), target.cuda()
        text_data, img_feat, target = Variable(text_data), Variable(img_feat), Variable(target)
        optimizer.zero_grad()
        output = model(text_data)
        loss = torch.sum(target * F.pairwise_distance(output, img_feat).view(-1))
        batch_loss =  loss/target.size(0)
        batch_loss.backward()
        optimizer.step()
        train_loss += loss.data[0]
        
    train_loss /= len(dataloader.dataset)
    print('\nTrain epoch: {} Average loss: {:.4f}\n'.format(
        epoch, train_loss))

In [ ]:
def evaluate(model, dataloader, batch_size = 128, cuda = False):
    

In [13]:
import time

for epoch in range(1, 100):
    start = time.time()
    train(epoch, model, dataloader_train_captions, captions_batch_size, 
          cuda = torch.cuda.is_available())
    train(epoch, model, dataloader_train_questions, questions_batch_size, 
          cuda = torch.cuda.is_available())
    print('Time taken: {}\n'.format(time.time() - start))

Process Process-4:
Process Process-3:
Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/andrewsklyar/miniconda3/envs/py-nlp1/lib/python3.5/multiprocessing/process.py", line 93,

KeyboardInterrupt: 

In [13]:
import h5py
import json
import numpy as np
import os
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from torch.utils.data import Dataset, DataLoader
import torch
import pickle

DEBUG = True
CUDA = torch.cuda.is_available()

torch.manual_seed(1)

captions_batch_size = 128
questions_batch_size = 512

img_data = './data/img_data'
text_data = './data/text_data'
img_feat_file = 'IR_image_features.h5',
img_map_file = 'IR_img_features2id.json',
complexity = 'easy'

json_train_file = os.path.join(text_data,
                               '{}/IR_train_{}.json'.format(complexity.capitalize(), complexity)
                               )
pickle_captions_train_file = os.path.join(text_data,
                                          '{}/IR_train_{}_captions.pkl'.format(complexity.capitalize(), complexity)
                                          )
pickle_questions_train_file = os.path.join(text_data,
                                           '{}/IR_train_{}_questions.pkl'.format(complexity.capitalize(), complexity)
                                           )

pickle_file_exists = os.path.isfile(pickle_captions_train_file) and os.path.isfile(pickle_questions_train_file)

stop = set(stopwords.words('english'))
stop.remove('no')
stop.update(['?', '\'s', ',', '!', '.', '-', ':', '_', ''])

stemmer = SnowballStemmer("english")
stem = True

In [14]:
csv_file = '{}/IR_{}_{}.json'.format(complexity.capitalize(), split, complexity)
csv_path = os.path.join(text_data, csv_file)

In [15]:
df = pd.read_json(csv_path).T.sort_index().head(5)

In [16]:
df.caption = df.caption.apply(lambda sentence:
        tuple(
            [(lambda x: stemmer.stem(x) if stem else x)(word)
             for word in word_tokenize(sentence.lower())
             if word not in stop or not stopwords]
        )
        )

0       (small, dog, pink, shirt, sit, gym, remot)
1    (hard, wood, floor, kitchen, black, applianc)
2                 (skii, slope, skier, ski, slope)
3          (man, hors, drawn, carriag, red, light)
4             (dog, bandana, glass, stare, someth)
Name: caption, dtype: object

In [18]:
df

,caption,dialog,img_list,target,target_img_id
0,a small dog with a pink shirt sitting gym a re...,"[[is the dog inside a gym ? no], [what kind of...","[298299, 359302, 442687, 289895, 66976, 51606,...",4,66976
1,a hard wood floor kitchen with black appliances,"[[can you see the stove ? yes], [how about the...","[576290, 258000, 53232, 335107, 112604, 317458...",1,258000
2,a skii slope with skiers skiing down the slope,"[[how many skiers ? there are 2], [male or fem...","[578041, 354874, 553671, 258753, 28093, 255500...",1,354874
3,a man in a horse drawn carriage at a red light,"[[are they on street ? yes], [do you see any p...","[64616, 268317, 48288, 373101, 465045, 422026,...",8,421024
4,a dog with bandana and glasses staring at some...,[[what color is the bandana ? blue with black ...,"[126296, 62231, 153888, 175121, 292574, 580837...",7,132331


In [ ]:
def text2int(self, text, vocab):
    return [vocab[word] for word in text]

In [ ]:
def questions2int(questions, vocab):
    questions = [sentence for sentences in text for sentence in questions]
    questions_int = []
    for question in questions:
        answer = question.split('?')[1].strip().lower()
        is_binary = answer == 'yes' or answer == 'no'
        if is_binary:
            processed_question = tuple([(lambda x: stemmer.stem(x) if stem else x)(word)
                                        for word in word_tokenize(question.lower())
                                        if word not in stop or not stopwords]
                                      )
            questions = text2int(processed_question)
            questions_int.append(questions)
    

In [17]:
df.dialog.apply(lambda arr:
               for sentence in arr:)

,caption,dialog,img_list,target,target_img_id
0,a small dog with a pink shirt sitting gym a re...,"[[is the dog inside a gym ? no], [what kind of...","[298299, 359302, 442687, 289895, 66976, 51606,...",4,66976
1,a hard wood floor kitchen with black appliances,"[[can you see the stove ? yes], [how about the...","[576290, 258000, 53232, 335107, 112604, 317458...",1,258000
2,a skii slope with skiers skiing down the slope,"[[how many skiers ? there are 2], [male or fem...","[578041, 354874, 553671, 258753, 28093, 255500...",1,354874
3,a man in a horse drawn carriage at a red light,"[[are they on street ? yes], [do you see any p...","[64616, 268317, 48288, 373101, 465045, 422026,...",8,421024
4,a dog with bandana and glasses staring at some...,[[what color is the bandana ? blue with black ...,"[126296, 62231, 153888, 175121, 292574, 580837...",7,132331
